In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
import os
import shutil
import cv2

In [ ]:
renamed_images_path='dataset/renamed_images'
smote_images_path='dataset/smote_images'
if os.path.isdir(smote_images_path):
    shutil.rmtree(smote_images_path)
os.makedirs(smote_images_path)
for image_class in os.listdir(renamed_images_path):
    for image in os.listdir(os.path.join(renamed_images_path, image_class)):
        image_path = os.path.join(renamed_images_path, image_class, image)
        try:
            img = cv2.imread(image_path)
            cv2.imwrite(os.path.join(smote_images_path, image), img)
        except Exception as e:
            print(e)
print('Images are saved in smote_images')

In [ ]:
def load_images_from_dir(directory):
    images = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):  # Change to your image format
            img = load_img(os.path.join(directory, filename), target_size=(224, 224))  # Resize as needed
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(filename.split("_")[0])  # Assuming filename is "class_imageNumber.jpg"

    return np.array(images), labels

image_directory = "dataset/smote_images/"
images, labels = load_images_from_dir(image_directory)

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)

In [ ]:
unique_classes, counts = np.unique(y_train, return_counts=True)

plt.bar(unique_classes, counts)
plt.xticks(unique_classes, label_encoder.inverse_transform(unique_classes))
plt.xlabel("Class")
plt.ylabel("Count")
plt.title("Class Distribution Before SMOTE")
plt.show()

In [ ]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train.reshape(-1, 224 * 224 * 3), y_train)

In [ ]:
# Count the occurrences of each class
unique_classes, counts = np.unique(y_train_resampled, return_counts=True)

plt.bar(unique_classes, counts)
plt.xticks(unique_classes, label_encoder.inverse_transform(unique_classes))  # Convert back to class names
plt.xlabel("Class")
plt.ylabel("Count")
plt.title("Class Distribution after SMOTE")
plt.show()

In [ ]:
output_directory = "dataset/smote_generated_images"

for i, (image, label) in enumerate(zip(X_train_resampled, y_train_resampled)):
    image = image.reshape(224, 224, 3)
    label_name = label_encoder.inverse_transform([label])[0]
    label_directory = os.path.join(output_directory, label_name)

    if not os.path.exists(label_directory):
        os.makedirs(label_directory)

    img = array_to_img(image)
    img.save(os.path.join(label_directory, f"synthetic_{i}.jpg"))